<a href="https://colab.research.google.com/github/francobarrionuevoenv21/codeforenviroprojects/blob/testGEE/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install earthengine-api

     -------------------------------------- 244.3/244.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 110.2/110.2 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 10.7/10.7 MB 9.2 MB/s eta 0:00:00
     ------------------------------------ 177.0/177.0 kB 761.7 kB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 120.3/120.3 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 77.4/77.4 kB 615.7 kB/s eta 0:00:00
  Using cached googleapis_common_protos-1.57.0-py2.py3-none-any.whl (217 kB)
     -------------------------------------- 527.0/527.0 kB 2.5 MB/s eta 0:00:00
  Created wheel for earthengine-api: filename=earthengine_api-0.1.335-py3-none-any.whl size=273772 sha256=2ed082f057095937034fcfd86481c2a910af2cc5bc6fbb92c9ff81a7b936a4f8
  Stored i

In [ ]:
# the regulars
import pandas as pd
import geopandas as gpd
import numpy as np

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWgavddNm_nGeJ8ekgVGBeQrj_SAU0xasBc0ZjqlOBcOybl72sH-kVz_h6s

Successfully saved authorization token.


In [ ]:
# Coordenadas 2018: -32.81910507118151, -60.42389548016447
# in decimal degrees
lat =  -32.81910507118151
lon = -60.42389548016447


# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# Landsat-8 takes 8 days to come back to the same place

# start date of range to filter for
start_date = '2020-08-01'

# end date
end_date = '2020-09-10'

# get the satellite data
Landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

# put the images in a list
landsat_list = Landsat8.toList(Landsat8.size());

for i in range(Landsat8.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

    print(date, cloud)

2020-08-04 1.18
2020-08-20 0.42
2020-09-05 23.4
2020-08-11 5.27
2020-08-27 21.47


In [ ]:

parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 600, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

display(Image(url = ee.Image(landsat_list.get(1)).getThumbUrl(parameters)))

In [ ]:
#Landsat8.size()
#print(landsat_list)

In [ ]:
print(ee.Image(landsat_list.get(1)).bandNames().getInfo())

print(ee.Image(landsat_list.get(1)).select('SR_B1').projection().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
{'type': 'Projection', 'crs': 'EPSG:32620', 'transform': [30, 0, 528585, 0, -30, -3554085]}


In [ ]:
# Se carga la imagen .shp correspondiente a los ROIs a partir del directorio
#rois_gdp = gpd.read_file('D:\TELEDETECCION\Proyecto_incendios_Delta\Muestras_2018-08-22 32620\Muestras_2018-08-22_32620.shp')#.set_index('id') #Levanta archivo shp
rois_gdp = gpd.read_file('D:\TELEDETECCION\Proyecto_incendios_Delta\Muestras_2018-08-22.shp')#.set_index('id') #Levanta archivo shp

# print(rois_gdp.geometry.values[0])

#geometry = rois_gdp.geometry.values[0]
rois_gdp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
rois_gdp_exp = rois_gdp.explode(index_parts = False)

In [ ]:
rois_gdp_exp.geometry[0]

TypeError: 'Polygon' object is not subscriptable

In [ ]:
#https://stackoverflow.com/questions/4741243/how-to-pick-just-one-item-from-a-generator
rois_gdp_iterf = rois_gdp.iterfeatures()
geom_data = next(rois_gdp_iterf)['geometry']['coordinates']

In [ ]:
list_lon = []
list_lat = []

for i in range(len(geom_data[0])):
    list_lon.append(geom_data[0][i][0])
    list_lat.append(geom_data[0][i][1])

In [ ]:
list_coord = []

for i in range(len(geom_data[0])):
    list_coord.append(list(geom_data[0][i]))

list_coord

[[-60.43470604358266, -32.834308101686894],
 [-60.36500181902507, -32.834154378659115],
 [-60.359696248126994, -32.95597310931026],
 [-60.43049817700832, -32.956740665506416],
 [-60.43470604358266, -32.834308101686894]]

In [ ]:
geom_clip = ee.Geometry.Polygon(list_coord)

In [ ]:
geom_clip

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -60.43470604358266,
              -32.834308101686894
            ],
            [
              -60.36500181902507,
              -32.834154378659115
            ],
            [
              -60.359696248126994,
              -32.95597310931026
            ],
            [
              -60.43049817700832,
              -32.956740665506416
            ],
            [
              -60.43470604358266,
              -32.834308101686894
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [ ]:
#ee.Image(landsat_list.get(1)).clip(geometry)

display(Image(url = ee.Image(landsat_list.get(1)).clip(geom_clip).getThumbUrl(parameters)))

Export image

References:
https://eamena.org/sites/default/files/eamena/documents/media/gis_4_-_introduction_to_gee_and_sentinel-2_imagery-compressed.pdf
https://gis.stackexchange.com/questions/300344/exported-bands-must-have-compatible-data-types-found-inconsistent-types-uint16
https://gis.stackexchange.com/questions/347340/export-google-earth-engine-landsat-image-to-google-drive-not-working-in-python-a

In [ ]:
select_bands = ee.Image(landsat_list.get(1)).bandNames().getInfo()[0:6]
select_bands

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6']

In [ ]:
image_bands = ee.Image(landsat_list.get(1)).select(select_bands)

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image = image_bands,
    region = geom_clip,
    #description = 'test01'
)

In [ ]:
export_task.start()

In [ ]:
export_task.status()

{'state': 'READY',
 'description': 'myExportImageTask',
 'creation_timestamp_ms': 1672542935978,
 'update_timestamp_ms': 1672542935978,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'SOGVC6ZNDRHMGGGMWYYPFWHM',
 'name': 'projects/earthengine-legacy/operations/SOGVC6ZNDRHMGGGMWYYPFWHM'}